In [1]:
%pip install tensorflow_datasets
%pip install tensorflow

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 48.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import KFold

In [4]:
import sys
("Python version is", sys.version)
import sklearn
print("Scikit-learn version is: ", sklearn.__version__)
import tensorflow as tf
print("Tensorflow version is: ", tf.__version__)

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

Scikit-learn version is:  1.4.1.post1
Tensorflow version is:  2.16.1


Case Ketika shuffle=False, random_state=None

In [5]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=None)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # Save the model
    model_save_path = f"models/model_fold_{fold_no}.h5"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    accuracy = accuracy_score(Y_test, y_pred_binary)
    acc_per_fold.append(accuracy)

    fold_no += 1

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

Found 240 files belonging to 60 classes.


Training for fold 1


2024-04-06 09:36:26.095543: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 521ms/step - accuracy: 0.0165 - loss: 0.8281 - val_accuracy: 0.0513 - val_loss: 0.1797
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.0878 - loss: 0.1848 - val_accuracy: 0.0513 - val_loss: 0.1426
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.1265 - loss: 0.1433 - val_accuracy: 0.0769 - val_loss: 0.1311
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step - accuracy: 0.2743 - loss: 0.1132 - val_accuracy: 0.1795 - val_loss: 0.1222
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.4124 - loss: 0.0796 - val_accuracy: 0.2308 - val_loss: 0.1088
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.5085 - loss: 0.0532 - val_accuracy: 0.2821 - val_loss: 0.0952
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 182ms/step - accuracy: 0.6885 - loss: 0.0409 - val_accuracy: 0.4615 - val_loss: 0.0859
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.834

Model saved to models/model_fold_1.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 2
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 530ms/step - accuracy: 0.0131 - loss: 0.9207 - val_accuracy: 0.0000e+00 - val_loss: 0.1904
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.0991 - loss: 0.1660 - val_accuracy: 0.2051 - val_loss: 0.1539
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.2751 - loss: 0.1294 - val_accuracy: 0.1282 - val_loss: 0.1476
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.3360 - loss: 0.0998 - val_accuracy: 0.2051 - val_loss: 0.1076
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.4751 - loss: 0.0628 - val_accuracy: 0.4103 - val_loss: 0.0784
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.6828 - loss: 0.0346 - val_accuracy: 0.6410 - val_loss: 0.0581
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.9005 - loss: 0.0216 - val_accuracy: 0.7949 - val_loss: 0.0498
Epoch 8/10
5/5 ━━━━━

...

Model saved to models/model_fold_3.h5


1/2 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7feb7a8170a0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 4
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step - accuracy: 0.0507 - loss: 0.6302 - val_accuracy: 0.0000e+00 - val_loss: 0.1701
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.0737 - loss: 0.1606 - val_accuracy: 0.0263 - val_loss: 0.1759
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy: 0.1470 - loss: 0.1344 - val_accuracy: 0.1316 - val_loss: 0.1553
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.3599 - loss: 0.0969 - val_accuracy: 0.2632 - val_loss: 0.1286
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.5941 - loss: 0.0635 - val_accuracy: 0.3947 - val_loss: 0.1050
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.7736 - loss: 0.0396 - val_accuracy: 0.4737 - val_loss: 0.0799
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.8690 - loss: 0.0240 - val_accuracy: 0.6053 - val_loss: 0.0624
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accura

Model saved to models/model_fold_4.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 5
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 496ms/step - accuracy: 0.0000e+00 - loss: 0.7682 - val_accuracy: 0.0526 - val_loss: 0.1748
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.0967 - loss: 0.1594 - val_accuracy: 0.1053 - val_loss: 0.2064
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.2979 - loss: 0.1454 - val_accuracy: 0.2105 - val_loss: 0.2002
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.4991 - loss: 0.0891 - val_accuracy: 0.3684 - val_loss: 0.1633
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.7849 - loss: 0.0496 - val_accuracy: 0.4737 - val_loss: 0.1320
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.7853 - loss: 0.0391 - val_accuracy: 0.5263 - val_loss: 0.1085
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.9280 - loss: 0.0151 - val_accuracy: 0.7105 - val_loss: 0.0889
Epoch 8/10
5/5 ━━━━━

Model saved to models/model_fold_5.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Accuracy for fold 1: 0.3958333333333333
Accuracy for fold 2: 0.6458333333333334
Accuracy for fold 3: 0.6041666666666666
Accuracy for fold 4: 0.7083333333333334
Accuracy for fold 5: 0.75


Case Ketika shuffle=False, random_state=None

In [6]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=None)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=False, random_state=None)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # Save the model
    model_save_path = f"models/model_fold_{fold_no}.h5"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    accuracy = accuracy_score(Y_test, y_pred_binary)
    acc_per_fold.append(accuracy)

    fold_no += 1

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

Found 240 files belonging to 60 classes.


Training for fold 1


2024-04-06 09:40:07.204000: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step - accuracy: 0.0282 - loss: 0.7471 - val_accuracy: 0.0513 - val_loss: 0.1168
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.0648 - loss: 0.1393 - val_accuracy: 0.1026 - val_loss: 0.1252
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.2104 - loss: 0.1180 - val_accuracy: 0.1538 - val_loss: 0.1185
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.4289 - loss: 0.0858 - val_accuracy: 0.2564 - val_loss: 0.1027
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.6267 - loss: 0.0533 - val_accuracy: 0.3846 - val_loss: 0.0789
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.7657 - loss: 0.0373 - val_accuracy: 0.5128 - val_loss: 0.0560
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.8670 - loss: 0.0231 - val_accuracy: 0.6410 - val_loss: 0.0424
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.9452 - loss: 0.0178 - val_accuracy: 0.7436 - val_loss:

Model saved to models/model_fold_1.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 2
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 477ms/step - accuracy: 0.0000e+00 - loss: 0.7484 - val_accuracy: 0.0256 - val_loss: 0.1702
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - accuracy: 0.0557 - loss: 0.1891 - val_accuracy: 0.1026 - val_loss: 0.1856
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.2608 - loss: 0.1459 - val_accuracy: 0.3077 - val_loss: 0.1436
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.5380 - loss: 0.0961 - val_accuracy: 0.3846 - val_loss: 0.0998
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.5885 - loss: 0.0591 - val_accuracy: 0.4872 - val_loss: 0.0770
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.8479 - loss: 0.0304 - val_accuracy: 0.5897 - val_loss: 0.0649
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.9187 - loss: 0.0190 - val_accuracy: 0.7436 - val_loss: 0.0486
Epoch 8/10
5/5 ━━━━━

Model saved to models/model_fold_2.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 3
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 480ms/step - accuracy: 0.0130 - loss: 0.8194 - val_accuracy: 0.0000e+00 - val_loss: 0.1548
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.0616 - loss: 0.1602 - val_accuracy: 0.0263 - val_loss: 0.1501
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.1630 - loss: 0.1455 - val_accuracy: 0.1579 - val_loss: 0.1323
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.3920 - loss: 0.0867 - val_accuracy: 0.3684 - val_loss: 0.1145
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.6192 - loss: 0.0536 - val_accuracy: 0.3947 - val_loss: 0.1056
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.7714 - loss: 0.0318 - val_accuracy: 0.5526 - val_loss: 0.0979
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.9016 - loss: 0.0224 - val_accuracy: 0.5526 - val_loss: 0.0950
Epoch 8/10
5/5 ━━━━━

Model saved to models/model_fold_3.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 4
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 469ms/step - accuracy: 0.0364 - loss: 0.9937 - val_accuracy: 0.0000e+00 - val_loss: 0.1885
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 198ms/step - accuracy: 0.0251 - loss: 0.1706 - val_accuracy: 0.0789 - val_loss: 0.1711
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.1180 - loss: 0.1588 - val_accuracy: 0.0789 - val_loss: 0.1501
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.2467 - loss: 0.1238 - val_accuracy: 0.1842 - val_loss: 0.1201
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.4887 - loss: 0.0771 - val_accuracy: 0.5526 - val_loss: 0.0958
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.6829 - loss: 0.0543 - val_accuracy: 0.6579 - val_loss: 0.0860
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.7992 - loss: 0.0356 - val_accuracy: 0.7105 - val_loss: 0.0711
Epoch 8/10
5/5 ━━━━━

Model saved to models/model_fold_4.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 5
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 463ms/step - accuracy: 0.0130 - loss: 0.6912 - val_accuracy: 0.0526 - val_loss: 0.1704
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.0776 - loss: 0.1796 - val_accuracy: 0.1053 - val_loss: 0.1568
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - accuracy: 0.1817 - loss: 0.1656 - val_accuracy: 0.0789 - val_loss: 0.1532
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.3529 - loss: 0.1037 - val_accuracy: 0.2368 - val_loss: 0.1315
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.5485 - loss: 0.0747 - val_accuracy: 0.2368 - val_loss: 0.1037
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.7203 - loss: 0.0440 - val_accuracy: 0.4474 - val_loss: 0.0848
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.8157 - loss: 0.0288 - val_accuracy: 0.5789 - val_loss: 0.0696
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_5.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Accuracy for fold 1: 0.6666666666666666
Accuracy for fold 2: 0.7916666666666666
Accuracy for fold 3: 0.6875
Accuracy for fold 4: 0.6458333333333334
Accuracy for fold 5: 0.6875


Case Ketika shuffle=True, random_state=42

In [7]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # Save the model
    model_save_path = f"models/model_fold_{fold_no}.h5"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    accuracy = accuracy_score(Y_test, y_pred_binary)
    acc_per_fold.append(accuracy)

    fold_no += 1

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

Found 240 files belonging to 60 classes.


Training for fold 1


2024-04-06 09:42:58.996397: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step - accuracy: 0.0365 - loss: 0.8944 - val_accuracy: 0.0513 - val_loss: 0.2645
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.0274 - loss: 0.1846 - val_accuracy: 0.0513 - val_loss: 0.2342
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.1386 - loss: 0.1505 - val_accuracy: 0.1026 - val_loss: 0.2197
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.2352 - loss: 0.1166 - val_accuracy: 0.2051 - val_loss: 0.1811
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.4372 - loss: 0.0783 - val_accuracy: 0.2308 - val_loss: 0.1432
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.5398 - loss: 0.0612 - val_accuracy: 0.4103 - val_loss: 0.1088
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 169ms/step - accuracy: 0.6715 - loss: 0.0425 - val_accuracy: 0.4615 - val_loss: 0.0849
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.8557 - loss: 0.0267 - val_accuracy: 0.5641 - val_loss:

Model saved to models/model_fold_1.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 2
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 472ms/step - accuracy: 0.0204 - loss: 0.8061 - val_accuracy: 0.0513 - val_loss: 0.1472
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.0782 - loss: 0.1584 - val_accuracy: 0.0769 - val_loss: 0.1553
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.1443 - loss: 0.1391 - val_accuracy: 0.1795 - val_loss: 0.1454
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.3277 - loss: 0.1157 - val_accuracy: 0.2564 - val_loss: 0.1117
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.4716 - loss: 0.0773 - val_accuracy: 0.3590 - val_loss: 0.0779
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - accuracy: 0.6176 - loss: 0.0520 - val_accuracy: 0.4615 - val_loss: 0.0555
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.7849 - loss: 0.0310 - val_accuracy: 0.6410 - val_loss: 0.0416
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_2.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Training for fold 3
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 482ms/step - accuracy: 0.0147 - loss: 0.8699 - val_accuracy: 0.0263 - val_loss: 0.1832
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.0685 - loss: 0.1763 - val_accuracy: 0.0263 - val_loss: 0.1708
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.1570 - loss: 0.1466 - val_accuracy: 0.2632 - val_loss: 0.1427
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.4119 - loss: 0.1007 - val_accuracy: 0.3421 - val_loss: 0.0979
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.6041 - loss: 0.0511 - val_accuracy: 0.5263 - val_loss: 0.0597
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.7675 - loss: 0.0334 - val_accuracy: 0.5789 - val_loss: 0.0447
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.8699 - loss: 0.0206 - val_accuracy: 0.6842 - val_loss: 0.0397
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_3.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 4
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.0260 - loss: 0.8599 - val_accuracy: 0.0000e+00 - val_loss: 0.2552
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - accuracy: 0.0126 - loss: 0.2101 - val_accuracy: 0.0263 - val_loss: 0.1576
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step - accuracy: 0.0603 - loss: 0.1268 - val_accuracy: 0.1316 - val_loss: 0.1584
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.2693 - loss: 0.1087 - val_accuracy: 0.2632 - val_loss: 0.1365
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - accuracy: 0.4132 - loss: 0.0778 - val_accuracy: 0.3421 - val_loss: 0.1085
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.6396 - loss: 0.0541 - val_accuracy: 0.4474 - val_loss: 0.0836
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.7866 - loss: 0.0341 - val_accuracy: 0.4737 - val_loss: 0.0645
Epoch 8/10
5/5 ━━━━━

Model saved to models/model_fold_4.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 5
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.0052 - loss: 0.7166 - val_accuracy: 0.0263 - val_loss: 0.1680
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.0832 - loss: 0.1592 - val_accuracy: 0.0789 - val_loss: 0.1838
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.2012 - loss: 0.1419 - val_accuracy: 0.1053 - val_loss: 0.1767
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 171ms/step - accuracy: 0.3907 - loss: 0.1027 - val_accuracy: 0.2105 - val_loss: 0.1560
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.5919 - loss: 0.0692 - val_accuracy: 0.3158 - val_loss: 0.1281
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.7259 - loss: 0.0403 - val_accuracy: 0.3684 - val_loss: 0.0881
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.8495 - loss: 0.0248 - val_accuracy: 0.5263 - val_loss: 0.0586
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_5.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Accuracy for fold 1: 0.3958333333333333
Accuracy for fold 2: 0.3958333333333333
Accuracy for fold 3: 0.6666666666666666
Accuracy for fold 4: 0.4166666666666667
Accuracy for fold 5: 0.5208333333333334


Case Ketika shuffle=True, random_state=123

In [8]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # Save the model
    model_save_path = f"models/model_fold_{fold_no}.h5"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred >= 0.5).astype(int)
    accuracy = accuracy_score(Y_test, y_pred_binary)
    acc_per_fold.append(accuracy)

    fold_no += 1

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

Found 240 files belonging to 60 classes.


Training for fold 1


2024-04-06 09:50:14.249856: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 465ms/step - accuracy: 0.0330 - loss: 0.9767 - val_accuracy: 0.0256 - val_loss: 0.1889
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.0652 - loss: 0.1677 - val_accuracy: 0.0000e+00 - val_loss: 0.1453
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.1930 - loss: 0.1157 - val_accuracy: 0.1538 - val_loss: 0.1323
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.4064 - loss: 0.0878 - val_accuracy: 0.3077 - val_loss: 0.1148
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.6002 - loss: 0.0559 - val_accuracy: 0.3846 - val_loss: 0.0959
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.7145 - loss: 0.0397 - val_accuracy: 0.4359 - val_loss: 0.0731
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 170ms/step - accuracy: 0.7871 - loss: 0.0262 - val_accuracy: 0.6154 - val_loss: 0.0589
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 166ms/step - accuracy: 0.9439 - loss: 0.0167 - val_accuracy: 0.6667 - val_l

Model saved to models/model_fold_1.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 2
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 475ms/step - accuracy: 0.0113 - loss: 0.7254 - val_accuracy: 0.0000e+00 - val_loss: 0.2055
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.0656 - loss: 0.1668 - val_accuracy: 0.0000e+00 - val_loss: 0.1394
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.1795 - loss: 0.1266 - val_accuracy: 0.1026 - val_loss: 0.1337
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - accuracy: 0.3503 - loss: 0.1028 - val_accuracy: 0.1538 - val_loss: 0.1086
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - accuracy: 0.4268 - loss: 0.0665 - val_accuracy: 0.2821 - val_loss: 0.0861
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.7010 - loss: 0.0424 - val_accuracy: 0.4103 - val_loss: 0.0714
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.7784 - loss: 0.0308 - val_accuracy: 0.5128 - val_loss: 0.0633
Epoch 8/10
5/5 ━

Model saved to models/model_fold_2.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 3
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 485ms/step - accuracy: 0.0330 - loss: 0.8833 - val_accuracy: 0.0263 - val_loss: 0.1457
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.0646 - loss: 0.1512 - val_accuracy: 0.0789 - val_loss: 0.1540
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step - accuracy: 0.1669 - loss: 0.1644 - val_accuracy: 0.2105 - val_loss: 0.1399
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - accuracy: 0.4072 - loss: 0.1130 - val_accuracy: 0.2895 - val_loss: 0.1178
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.5507 - loss: 0.0745 - val_accuracy: 0.4474 - val_loss: 0.0990
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.6956 - loss: 0.0450 - val_accuracy: 0.4737 - val_loss: 0.0933
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.7775 - loss: 0.0366 - val_accuracy: 0.5789 - val_loss: 0.0843
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_3.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 4
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 469ms/step - accuracy: 0.0069 - loss: 0.7984 - val_accuracy: 0.0263 - val_loss: 0.1817
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 168ms/step - accuracy: 0.0798 - loss: 0.1632 - val_accuracy: 0.0789 - val_loss: 0.1441
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 0.2181 - loss: 0.1282 - val_accuracy: 0.1842 - val_loss: 0.1059
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 172ms/step - accuracy: 0.4983 - loss: 0.0850 - val_accuracy: 0.3158 - val_loss: 0.0775
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step - accuracy: 0.5858 - loss: 0.0622 - val_accuracy: 0.6053 - val_loss: 0.0465
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - accuracy: 0.7879 - loss: 0.0327 - val_accuracy: 0.6579 - val_loss: 0.0351
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.8686 - loss: 0.0233 - val_accuracy: 0.6579 - val_loss: 0.0303
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_4.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Training for fold 5
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 481ms/step - accuracy: 0.0078 - loss: 0.7799 - val_accuracy: 0.0526 - val_loss: 0.1339
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 176ms/step - accuracy: 0.0685 - loss: 0.1566 - val_accuracy: 0.1053 - val_loss: 0.1444
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - accuracy: 0.1726 - loss: 0.1431 - val_accuracy: 0.1316 - val_loss: 0.1273
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.3221 - loss: 0.1225 - val_accuracy: 0.2368 - val_loss: 0.0992
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.5503 - loss: 0.0744 - val_accuracy: 0.2895 - val_loss: 0.0714
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.6474 - loss: 0.0557 - val_accuracy: 0.4737 - val_loss: 0.0514
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 184ms/step - accuracy: 0.7931 - loss: 0.0368 - val_accuracy: 0.6579 - val_loss: 0.0400
Epoch 8/10
5/5 ━━━━━━━━━

Model saved to models/model_fold_5.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Accuracy for fold 1: 0.5416666666666666
Accuracy for fold 2: 0.5416666666666666
Accuracy for fold 3: 0.6041666666666666
Accuracy for fold 4: 0.75
Accuracy for fold 5: 0.5208333333333334


Batch Size = 30

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Load your image dataset and preprocessing here
# Assume X is your image data and Y is your labels
# Modify this part according to your image dataset loading and preprocessing

# Define your image data (X) and labels (Y) here
# Example: X = ... # Load images using some method (e.g., tf.keras.preprocessing.image_dataset_from_directory)
X = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='rgb',  # Mengatur mode warna menjadi grayscale
    batch_size=30,
    image_size=(52, 52),
    shuffle=True,
    seed=123,
)
# Example: Y = ... # Load labels (may need to be encoded)
def extract_labels(image, label):
    return label

# Map the dataset to extract labels
Y = X.map(extract_labels)

# Split data into train and test sets
# Convert dataset to numpy arrays
X_list = []
Y_list = []
for images, labels in X.as_numpy_iterator():
    X_list.extend(images)
    Y_list.extend(labels)
X = np.array(X_list)
Y = np.array(Y_list)

# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# Convert labels to one-hot vectors
Y_train = to_categorical(Y_train, num_classes=60)
Y_test = to_categorical(Y_test, num_classes=60)

# Define the ResNet50 model
def build_resnet50_model():
    # Membuat base model ResNet50 tanpa lapisan Fully Connected terakhir
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(52, 52, 3),
    )

    # Menghentikan layer-layer yang ada pada base model agar tidak ter-update selama pelatihan
    for layer in base_model.layers:
        layer.trainable = False

    # Membangun model dengan menambahkan lapisan kustom di atas base model
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(60, activation='softmax')(x)

    # Membuat model gabungan
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

# Initialize k-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=123)

# Execute K-fold cross-validation
fold_no = 1
acc_per_fold = [] # Save accuracy from each fold

# Train the model for each split (fold)
for train_index, test_index in kf.split(X_train):
    print('Training for fold', fold_no)

    # Get train and test data for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[test_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[test_index]

    # Build the model
    model = build_resnet50_model()

    # Compile the model
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, Y_train_fold, batch_size=30, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    # Save the model
    model_save_path = f"models/model_fold_{fold_no}.h5"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # # Evaluate the model
    # y_pred = model.predict(X_test)
    # y_pred_binary = (y_pred >= 0.5).astype(int)
    # accuracy = accuracy_score(Y_test, y_pred_binary)
    # acc_per_fold.append(accuracy)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    true_classes = np.argmax(Y_test, axis=1)
    accuracy = accuracy_score(true_classes, y_pred_classes)
    acc_per_fold.append(accuracy)

    # Print predicted and true classes
    print("Predicted Classes:", y_pred_classes)
    print("True Classes:", true_classes)
    
    fold_no += 1

# Print accuracy for each fold
for i, acc in enumerate(acc_per_fold, 1):
    print(f'Accuracy for fold {i}: {acc}')

Found 240 files belonging to 60 classes.


Training for fold 1


2024-04-06 15:40:15.112750: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 375ms/step - accuracy: 0.0184 - loss: 0.8284 - val_accuracy: 0.0256 - val_loss: 0.1387
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.0602 - loss: 0.1548 - val_accuracy: 0.2308 - val_loss: 0.1244
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.2275 - loss: 0.1309 - val_accuracy: 0.2564 - val_loss: 0.1027
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.3926 - loss: 0.0872 - val_accuracy: 0.1538 - val_loss: 0.0957
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.5039 - loss: 0.0625 - val_accuracy: 0.3077 - val_loss: 0.0698
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.6972 - loss: 0.0416 - val_accuracy: 0.5128 - val_loss: 0.0602
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 132ms/step - accuracy: 0.8332 - loss: 0.0269 - val_accuracy: 0.5128 - val_loss: 0.0515
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.9133 - loss: 0.0214 - val_accuracy: 0.6667 - val_loss:

Model saved to models/model_fold_1.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Predicted Classes: [30 37 47 58 28 13 14  2 55 52 26 37 56 23 13 22 33 37 20 49 14  3 52 59
 13 19  3  0 48 31 58  3 13 13 17 44 53 26 13 36 24 16 32 35 40  3 30 44]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Training for fold 2
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 380ms/step - accuracy: 0.0296 - loss: 1.0594 - val_accuracy: 0.0000e+00 - val_loss: 0.2111
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.0371 - loss: 0.1987 - val_accuracy: 0.0513 - val_loss: 0.1484
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.0738 - loss: 0.1535 - val_accuracy: 0.0769 - val_loss: 0.1361
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.2017 - loss: 0.1104 - val_accuracy: 0.0769 - val_loss: 0.1104
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.3966 - 

Model saved to models/model_fold_2.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Predicted Classes: [30 37 47 37 28  6 14  2 55 52 30 57 11 23 37 22 33 37 50 49 14  8 37 59
 37 37  3  0 48 31 50  3  6 37  3 44 53 44  4 37 37 16 55 35 40  8 30 31]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Training for fold 3
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 375ms/step - accuracy: 0.0000e+00 - loss: 0.9411 - val_accuracy: 0.0000e+00 - val_loss: 0.1588
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.0423 - loss: 0.1556 - val_accuracy: 0.1053 - val_loss: 0.1476
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.1315 - loss: 0.1398 - val_accuracy: 0.1316 - val_loss: 0.1356
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.3480 - loss: 0.0981 - val_accuracy: 0.2895 - val_loss: 0.1201
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.546

Model saved to models/model_fold_3.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Predicted Classes: [53 36 47 39 28  6 14  2 55 52 26 36 11 23 20 22 33 37 20 49 14  8 38 20
 36 36 11  0 48 31  0 13  6 36 17 51 53 38 45 36 24 16 25 35 40 45 53 51]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Training for fold 4
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 473ms/step - accuracy: 0.0028 - loss: 0.8050 - val_accuracy: 0.0526 - val_loss: 0.1588
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.0587 - loss: 0.1547 - val_accuracy: 0.0526 - val_loss: 0.1409
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.1530 - loss: 0.1324 - val_accuracy: 0.0789 - val_loss: 0.1220
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.3243 - loss: 0.0827 - val_accuracy: 0.3158 - val_loss: 0.0846
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - accuracy: 0.5268 - loss

Model saved to models/model_fold_4.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step
Predicted Classes: [30 13 47  9 28  6 14  2 55 52 26 37 11 23 45 22 33 37 50 49 14 42 38 59
 13 19  3 32 48 10 50  3  6 13 17 44 53 38 13 27 24 16 32 35 40 40 30 51]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Training for fold 5
Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 375ms/step - accuracy: 0.0383 - loss: 0.8458 - val_accuracy: 0.0263 - val_loss: 0.1860
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.0573 - loss: 0.1656 - val_accuracy: 0.0789 - val_loss: 0.1608
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.2413 - loss: 0.1309 - val_accuracy: 0.1053 - val_loss: 0.1490
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - accuracy: 0.3570 - loss: 0.0971 - val_accuracy: 0.1053 - val_loss: 0.1171
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - accuracy: 0.4793 - loss

Model saved to models/model_fold_5.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Predicted Classes: [30 37 47 58 28 39 14  2 55 52 26 37 11 23 13 22 33 46 50 49 14 29 10 59
 13 19  3 12 48 31 50  3 39 37 17 44 53 38 37 37 21 16 32 35 40 45 30 51]
True Classes: [30  9 47 58 28  6 14  2 55 52 26 36 11 23 13 22 33 12 50 49 14  8 38 59
  9 19  3  0 48 31 50  3  6  9 17 44 53 38 13 36 24 16 32 35 40  8 30 51]
Accuracy for fold 1: 0.6875
Accuracy for fold 2: 0.6458333333333334
Accuracy for fold 3: 0.625
Accuracy for fold 4: 0.75
Accuracy for fold 5: 0.7083333333333334
